# Fourier Series

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact, IntSlider, FloatSlider

### Formula Definitions

If a time domain signal is periodic, i.e. $x(t) = x(t + nT)$ with $n \in \mathbb{Z}$, it can be expresed as an infinite series of harmonic exponentials, with other words, a **Fourier Series**:

\begin{align}
    x(t) = \sum_{k = -\infty}^{\infty} X_k \mathrm{e}^{j k \frac{2 \pi}{T}t}.
\end{align}

For $T$-periodic signals, only the isolated frequencies $\omega_k = k\frac{ 2 \pi}{T}$ with $k \in \mathbb{Z}$ are needed. As we will see, this results in a discrete spectrum.

By regrouping the terms and making use of Euler's identity $\mathrm{e}^{j \theta} = \cos(\theta) + j \sin(\theta)$, another more prominent form can be obtained:

\begin{align}
    x(t) = \frac{a_0}{2} + \sum_{k = 1}^{\infty} a_k \cos \left(k \frac{2 \pi}{T}t\right) + b_k \sin \left(k \frac{2 \pi}{T}t\right),
\end{align}

with the relationships

\begin{align}
    a_0 &= 2 X_0 \\
    a_k &= X_k + X_{-k} \\
    b_k &= j(X_k - X_{-k}).
\end{align}

The examples only consist of signals with an even symmetry, i.e. $x(t) = x(-t)$. 
The sine functions show odd symetry ($x(t) = -x(-t)$). Thus, they are not needed later, resulting in $b_k = 0 \ \forall  k$.

The Fourier coefficients can be computed by multiplication with the conjugated complex exponential and averaging over a single period:

\begin{align}
    X_k = \frac{1}{T} \int_{-T/2}^{T/2} x(t) \mathrm{e}^{-j k \frac{2 \pi}{T}t} \mathrm{d}t = \left. \left(x(t) \cdot \mathrm{e}^{-j k \frac{2 \pi}{T}t}\right) \ast \frac{1}{T}\mathrm{rect}\left(\frac{t}{T}\right) \ \right|_{t=0}.
\end{align}

In terms of electrical engineering, this can be coined as downmixing with $\mathrm{e}^{-j k \frac{2 \pi}{T}t}$, lowpass filtering with a filter impulse response $h(t) = \frac{1}{T}\mathrm{rect}\left(\frac{t}{T}\right)$ (thus, averaging over one period) and sampling the filter output at $t=0$.   
Another interpretation, rooted in linear algebra, interprets the formula as a scalar product:

\begin{align}
    X_k = \langle \mathrm{e}^{-j k \frac{2 \pi}{T}t}, x(t) \rangle,
\end{align}

i.e. the similarity of the time signal with a complex exponential of frequency $\omega_k = k \frac{2 \pi}{T}$ is computed. Thus, the input signal is **decomposed** into the (orthogonal) Fourier basis functions.

### Relationship with Fourier Transform
For periodically repeated base pulses $x_e(t)$, the coefficients can be obtained with the fourier transform $X_e(j \omega)$ of the base impulse:

\begin{align}
    x(t) = \sum_{n = -\infty}^{\infty} x_e(t- nT)
    = x_e(t) \ast \sum_{n= -\infty}^{\infty} \delta \left(t-nT\right).
\end{align}

In the follow-up, we need the transformation pair of the dirac comb:

\begin{align}
    \mathcal{F}\left\{ \sum_{n= -\infty}^{\infty} \delta \left(t-nT\right) \right\} = \frac{2 \pi}{T} \sum_{k= -\infty}^{\infty} \delta \left( \omega - k\frac{2 \pi}{T} \right).
\end{align}

Expoiting that convolution in time domain means multiplication in frequency domain and further exploiting the sampling property of the dirac distribution, we end up with:

\begin{align}
    X(j\omega) = \mathcal{F}\left\{ x(t) \right\} &=  X_e(j \omega) \ \frac{2 \pi}{T} \sum_{k= -\infty}^{\infty} \delta \left( \omega - k\frac{2 \pi}{T} \right) \\
    &=  \sum_{k= -\infty}^{\infty} \frac{1}{T}  X_e \left( jk \frac{2 \pi}{T}\right) 2 \pi \delta \left( \omega - k\frac{2 \pi}{T} \right).
\end{align}

Since $X_e \left( jk \frac{2 \pi}{T}\right)$ does not depend on $\omega$, the diracs can be transformed back separetely:

\begin{align}
    \mathcal{F}^{-1}\left\{2 \pi \delta(\omega - \omega_k) \right\} = \mathrm{e}^{j \omega_k t}.
\end{align}

Thus, in time domain, the **Fourier Series** 
\begin{align}
    x(t) = \sum_{k = -\infty}^{\infty} \frac{1}{T} X_e \left( jk \frac{2 \pi}{T}\right) \mathrm{e}^{j k \frac{2 \pi}{T}t}
\end{align}

remains and thus the relationship

\begin{align}
    X_k = \frac{1}{T} X_e \left( jk \frac{2 \pi}{T}\right).
\end{align}

holds.

## Joint Definitions for Visualizations

In [3]:
T = 3
L = 1.5
t = np.linspace(-L*T, L*T, 6000)

# number of coefficients used
K_max = 50
k_c = np.arange(K_max)
# Cosine kernel als matrix (for fourier series)
Cos = np.cos(2*np.pi/T*(t[:, None] @ k_c[None ,:]))
Cos[:, 0] = 0.5

FONT_SIZE = 16
TFONT_SIZE = 14

In [4]:
# function to plot a finite fourier series from the coefficients
# and the reference signal
def plot_fourier_series(a_k, x_ref, t, T):
    # Fouriersumme als Matrixprodukt
    x_c = Cos[:, :a_k.shape[0]] @ a_k
    
    plt.axhline(c='C7')
    plt.plot(t/T, x_ref, alpha=0.5, lw=3) # plotting square pulses
    plt.plot(t/T, x_c, 'r', lw=2);  #plotting cos karnels
    plt.axvline(c='C7')
    plt.grid(True);
    plt.ylabel('$x(t)$', fontsize=FONT_SIZE)
    plt.xlabel('$t/T$', fontsize=FONT_SIZE)
    plt.xticks(fontsize=TFONT_SIZE)
    plt.yticks(fontsize=TFONT_SIZE)
    plt.ylim((-0.2, 1.2))
    plt.xlim((-1.5, 1.5))

In [5]:
# function to plot a finite fourier series from the coefficients
# and the reference signal
def plot_fourier_spectrum(a_k, T):
    k = np.arange(-a_k.shape[0]+1, a_k.shape[0])
    X_k = np.concatenate([a_k[::-1]/2, a_k[1::]/2])
    plt.axhline(c='C7')
    (markerline, stemlines, baseline) = plt.stem(k, X_k)
    plt.setp(baseline, visible=False)
    plt.setp(markerline, linewidth=3)
    plt.setp(markerline, markersize=10)
    plt.axvline(c='C7')
    plt.grid(True);
    plt.ylabel('$X_k$', fontsize=FONT_SIZE)
    plt.xlabel('$k$', fontsize=FONT_SIZE)
    plt.xticks(fontsize=TFONT_SIZE)
    plt.yticks(fontsize=TFONT_SIZE)
    plt.ylim((-0.5, 1.2))
    plt.xlim((-50, 50))

In [6]:
# function to repeat a single pulse
# (signal must be time dependent)
def rep_pulse(signal, t, T):
    l = np.arange(-np.floor(L), np.floor(L)+1)
    return np.sum(signal(t[:,None] - l[None, :]*T), 1)

In [7]:
# Function to plot Fourier sum together with repeated pulse
def plot_signal(a_k, ref_pulse, t, T):
    x_ref = rep_pulse(ref_pulse, t , T)
    plt.figure(figsize=(16, 12))
    plt.subplot(211)
    plot_fourier_series(a_k, x_ref, t, T)
    plt.subplot(212)
    plot_fourier_spectrum(a_k, T)

#### Note:
The following sections use the duty cycle (Tastverhältnis) $D= \frac{T_e}{T}$ as known from power electronics. This denotes the fraction of a period where the pulse is on high level. e.g. for $D=\frac{T_e}{T} = 0.75$, the pulse sequence is on high level three fourths of a period.

## Fourier Series of a Rectangular Pulse sequence

A rectangular pulse sequence is the repetition of a single rectpulse:
\begin{align}
    x_1(t) = \sum_{n = -\infty}^{\infty} \mathrm{rect} \left(\frac{t-nT}{DT}\right)
    = \mathrm{rect} \left(\frac{t}{DT}\right) \ast \sum_{n= -\infty}^{\infty} \delta \left(t-nT\right)
\end{align}

Thus, $x_e(t) = \mathrm{rect}\left(\frac{t}{T D}\right)$.

It follows that $X_e(j \omega) = \mathcal{F}\left\{\mathrm{rect}\left(\frac{t}{T D}\right)\right\} = T D\ \mathrm{si}\left( \frac{\omega D T}{2} \right)$.

With the derivation above, $X_{1,k} = D\ \mathrm{si} \left( k D \pi \right)$ and with $a_k = X_k + X_{-k}$ it holds that

\begin{align}
\Rightarrow x_1(t) = \sum_{k= -\infty}^{\infty} D\ \mathrm{si} \left( k D \pi \right)\ \mathrm{e}^{j k \frac{2 \pi}{T} t}
= D + \sum_{k= 1}^{\infty} 2D\ \mathrm{si} \left( k D \pi \right) \mathrm{cos} \left(k \frac{2 \pi}{T} t \right)
\end{align}

In [8]:
# rectangular pulse
def rect(t, T):
    return 1.0*(np.abs(t/T) < 1/2) if T > 0 else t*0

In [9]:
@interact(K=IntSlider(min=1, max=K_max, step=1, value=10), 
          D=FloatSlider(min=0, max=1, step=0.05, value=0.5))
def plot_rect(K, D):
    k = np.arange(K)
    Rect_k = 2*D*np.sinc(k*D)
    # the lambda expression fixes the duration argument
    plot_signal(Rect_k, lambda t: rect(t, D*T), t, T)
    plt.ylim((-0.2, 1.1*D))

interactive(children=(IntSlider(value=10, description='K', max=50, min=1), FloatSlider(value=0.5, description=…

## Fourier series of a triangular pulse

The triangular pulse sequence can be described in a similar way:

\begin{align}
    x_2(t) = \sum_{n = -\infty}^{\infty} \Lambda \left(\frac{t-nT}{DT/2}\right)
    = \Lambda \left(\frac{t}{DT/2}\right) \ast \sum_{n= -\infty}^{\infty} \delta \left(t-nT\right)
\end{align}

Thus, $x_e(t) = \Lambda \left(\frac{t}{DT/2}\right)$ und es gilt $X_e(j\omega) = \mathcal{F}\left\{\Lambda \left(\frac{t}{DT/2}\right)\right\} = \frac{T D}{2} \mathrm{si}^2 \left( \frac{\omega D T}{4} \right)$

Therefore, $X_{2,k} = \frac{D}{2}\ \mathrm{si}^2 \left(\frac{k D}{2} \pi \right)$  and with $a_k = X_k + X_{-k}$ it holds that

\begin{align}
\Rightarrow x_2(t) = \sum_{k= -\infty}^{\infty} \frac{D}{2}\ \mathrm{si}^2 \left( \frac{k D}{2} \pi \right)\ \mathrm{e}^{j k \frac{2 \pi}{T} t}
= \frac{D}{2} + \sum_{k= 1}^{\infty} D\ \mathrm{si}^2 \left( \frac{k D}{2} \pi \right) \mathrm{cos} \left(k \frac{2 \pi}{T} t \right)
\end{align}

In [10]:
# triangular pulse:
def tri(t, T):
    return np.maximum(1-np.abs(t/T), 0) if T > 0 else t*0

In [11]:
@interact(K=IntSlider(min=1, max=K_max, step=1, value=4), 
          D=FloatSlider(min=0, max=1, step=0.05, value=0.5))
def plot_triag(K, D):
    k = np.arange(K)
    Tri_k = D*np.sinc(k*D/2)**2
    # the lambda expression fixes the duration argument
    plot_signal(Tri_k, lambda t: tri(t, T*D/2), t, T)
    plt.ylim((-0.1, 0.6))

interactive(children=(IntSlider(value=4, description='K', max=50, min=1), FloatSlider(value=0.5, description='…